In [10]:
from langchain_unstructured import UnstructuredLoader
from unstructured.cleaners.core import clean_extra_whitespace
import os

def list_rst_files(directory):
    rst_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            #print(file)
            if file.endswith('.rst'):
                rst_files.append(os.path.join(root, file))
    return rst_files

directory = '../EAFJupyter-master/docs' #just downloaded the zip of this repo https://github.com/mapsacosta/EAFJupyter/tree/master
print(list_rst_files(directory))
loader = UnstructuredLoader(list_rst_files(directory), post_processors=[clean_extra_whitespace])

00_comp_capabilities.rst
00_eaf_dae.rst
00_getting_started.rst
00_introduction.rst
00_jupyter_advanced.rst
00_jupyter_basics.rst
00_user_accounts.rst
01_acorn.rst
01_cmslpc.rst
01_cosmic.rst
01_dune_lbnf.rst
01_experiment_specific_accounts.rst
01_fermi_gen.rst
01_fife.rst
01_getting_fnal_accounts.rst
01_inference.rst
01_jobsub.rst
01_jupyterhub_extensions.rst
01_jupyterhub_terminal.rst
01_jupyterhub_terminal_howto.rst
01_managing_notebooks.rst
01_named_sessions.rst
01_new_users_python.rst
01_storage.rst
01_tips_tricks.rst
01_troubleshooting.rst
01_tutorials.rst
02_add_extensions.rst
02_batch_dask_cms.rst
02_cms_accounts.rst
02_coffea_examples.rst
02_common_issues.rst
02_contact.rst
02_customization.rst
02_dask_errors.rst
02_dask_extension.rst
02_dask_usage_jupyter.rst
02_fileops_jupyter.rst
02_fileops_xrootd.rst
02_github_extension.rst
02_important_links.rst
02_installing_libraries.rst
02_jupyter_console_ui.rst
02_local_dask.rst
02_nb_conda_kernels.rst
02_preinstalled_libs.rst
02_submi

In [11]:

docs = loader.load()

docs[0]
print(docs[0].metadata)

{'source': '../EAFJupyter-master/docs/00_comp_capabilities.rst', 'category_depth': 0, 'languages': ['cat'], 'file_directory': '../EAFJupyter-master/docs', 'filename': '00_comp_capabilities.rst', 'filetype': 'text/x-rst', 'last_modified': '2024-12-04T11:35:12', 'category': 'Title', 'element_id': '71d3b1ca98f65abe6fe97a1695bb8b30'}


In [13]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
LANGCHAIN_PROJECT="pr-oily-flash-79"

In [54]:
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore


#chat_model = ChatOllama(model="llama3.1:8b")
#embeddings = OllamaEmbeddings(model="llama3.1:8b")

chat_model = ChatOllama(model="qwen2.5-coder:14b")
embeddings = OllamaEmbeddings(model="qwen2.5-coder:14b")
vector_store = InMemoryVectorStore(embeddings)

#chat_model.invoke("Who was the first man on the moon?")

In [55]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [56]:
docs = loader.load()

In [58]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
# Index chunks

In [ ]:
_ = vector_store.add_documents(documents=all_splits)

In [35]:

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

In [40]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = chat_model.invoke(messages)
    return {"answer": response.content}

In [42]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [52]:
response = graph.invoke({"question": "tensorboard is not working for me. It has been stuck in \"initializing\" status for quite some time on the Elastic Analysis Facility?"})
#(response)
print(response["answer"])

INFO: HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO: HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


I don't know the specific cause of the issue with Tensorboard stuck on "initializing" status in the Elastic Analysis Facility. However, I can suggest checking the logs in the sandbox directory for any errors or issues that may be causing the problem. Additionally, you can try using condor commands to check on your workers and see if there are any issues with resource allocation.
